In [3]:
import pandas as pd
import numpy as np
import re
import codecs
from wordcloud import WordCloud
import jieba.analyse
import matplotlib.pyplot as plt

#************************************
#功能：读取QQ导出的txt聊天记录文件，并整理格式为pandas，保留信息：日期、时间、昵称、内容（去掉了QQ号）
#输入：文件路径，filepath
#输出：整理好的pandas对象 log_pd_output
#************************************
def log_txt_to_pd (filepath) :
    #step1 建立容器
    log_title_arr=[]      #聊天内容title行，包含发言时间、发言昵称、QQ号等
    log_content_arr=['n']  #聊天内容
    new_log_flag=0    #新聊天内容的标志位
    
    #step2 从txt文件中逐行读取，并判断是title还是content
    for word in open(filepath, 'r', encoding='utf-8'):
        tmp = word[0:5]
        if (tmp == "2019-"):  # 识别日期，但是若聊天记录内容中有同样开头的内容，将识别错误，不过概率很小
            log_title_arr.append(word)
            new_log_flag=1  #识别到日期，说明是新的一条聊天记录，标志位设置为1
            continue
        
        if new_log_flag==1: #如果标志位为1，说明此行文本是新聊天内容，追加到新记录
            log_content_arr.append(word)
            new_log_flag=0  #追加完要重置为0
        else:
            log_content_arr[-1]=log_content_arr[-1]+word   #标志位为0，说明此行文本是仍然是同一条聊天内容，拼接到同一条记录
    
    #step3 文本内容处理，格式调整    
    del log_content_arr[0]       #qq聊天记录的第一模块为文件头，多余信息，需要删掉
    ### 3.1将分好的title和content合并成df
    log_dict={"head" : log_title_arr,            #转为字典
           "content" :log_content_arr}  
    log_pd=pd.DataFrame(log_dict)                #转为panda
    ### 3.2Title内容处理，提取其中的日期、时间和玩家QQ昵称  例如：“2019-07-04 8:46:44 【潜水】省略号……(1031795834)”
    tmp1_split=log_pd['head'].str.split(' ',expand=True) #用空格分隔，分出日期、时间、昵称+QQ号
    tmp2_split=tmp1_split[2].str.split('(',expand=True) #用括号分出昵称    
    ### 3.3Content内容处理，剔除“@少年 ”这类的内容，例如：“@少年 你怎么不干活”（注意，是带空格的）
    atter_pat = '@(.*?) '   #构建艾特玩家的正则匹配，
    log_pd['content']=log_pd['content'].str.replace('@(.*?) ',' ').replace('\r','').replace('\n','') #去掉艾特内容，去掉\r之类的标记
    ### 3.4生成结果df
    log_pd_output=pd.DataFrame()
    log_pd_output['date']=tmp1_split[0]
    log_pd_output['time']=tmp1_split[1]
    log_pd_output['name']=tmp2_split[0]
    log_pd_output['content']=log_pd['content']
    
    return log_pd_output
 
#************************************
#功能：分词函数，调用jieba分词，并过滤停用词
#输入：需要分词的语句列表sentence，停用词文件路径stopword_filepath
#输出：分词后的长字符串 outstr
#************************************
def seg_sentence(sentence,stopword_filepath):
    sentence_seged=jieba.cut(str(sentence).strip())                                   #jieba初步分词
    stopwords=[line.strip() for line in open(stopword_filepath,'r').readlines()]      #加载停用词
    outstr=''                                                                         #创建输出容器
    for word in sentence_seged:                                                       #过滤掉所有停用词
        if word not in stopwords:
            if word!='\t':
                outstr+=word
                outstr+=' '
    return outstr

#***********主程序************
#1、解析聊天记录txt
qqlog=pd.DataFrame()
qqlog=log_txt_to_pd ('咔叽探险4队.txt')  #将聊天记录处理成df格式
qqlog.to_excel('咔叽4群(已整理).xlsx')  #导出预处理数据

#2、选定数据，去除干扰数据
qqlog_selected=qqlog[qqlog['date'].isin(['2019-07-05'])]   #选择需要的日期
qqlog_selected=qqlog_selected[~qqlog_selected['name'].isin(['【管理员】最萌叽气人JuJu[试用期]','最萌叽气人JuJu[试用期]'])]   #去掉机器人的发言

#3、分词
qqlog_content=qqlog_selected['content'].tolist()          #转为list
words=seg_sentence(qqlog_content,'qqun_stopwords.txt')    #调用停用词库进行分词

#4、进行词频分析
keywords = jieba.analyse.extract_tags(
    str(words), topK=50, withWeight=True, allowPOS=("n", "v"))   #调用jieba分词的词频分析功能
word_weights = pd.DataFrame(keywords)

#5、输出词频结果
word_weights.head(20)    


,0,1
0,虚空,0.177253
1,查询,0.113062
2,撤回,0.102290
3,传奇,0.095710
4,公测,0.078228
5,钻石,0.078134
6,攻略,0.077517
7,武器,0.068340
8,主线,0.067524
9,删档,0.066956


In [5]:
qqlog.head(50)

,date,time,name,content
0,2019-06-27,16:02:45,系统消息,你已经是群成员了，和大家打个招呼吧！
1,2019-06-27,16:02:46,JUJU,萌新守护者驾到！欢迎欢迎！(o*v*)ノ我是智障播报员JuJu，竭诚为您服务！想要了解更多...
2,2019-06-27,16:06:06,水狮提督,[图片]
3,2019-06-27,16:06:29,水狮提督,老阿姨把老板忘了
4,2019-06-27,16:06:44,水狮提督,到今天才进来
5,2019-06-27,16:10:00,げ怡葒Yè仴ゆ,[表情]
6,2019-06-27,16:10:12,げ怡葒Yè仴ゆ,背着老板小声逼逼~~
7,2019-06-27,16:10:35,少年Nick,大家好 我是老阿姨手下练习两年半的小弟
8,2019-06-27,16:19:06,男装老阿姨,[图片]
9,2019-06-27,16:20:05,苍白的正义,你会唱跳RAP篮球？
